In [1]:
import time

import torch

import torch.nn.functional as F
from torch.nn import ModuleList, Embedding
from torch.nn import Sequential, ReLU, Linear
from torch.nn import CrossEntropyLoss, MSELoss, L1Loss
from torch.optim.lr_scheduler import ReduceLROnPlateau

from torch_geometric.utils import degree
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, PNAConv, BatchNorm, global_add_pool

from phylognn_model import G2Dist_GCNConv

from gene_graph_dataset import GeneGraphDataset

from torch.utils.tensorboard import SummaryWriter

In [2]:
train_p, test_p = 0.7, 0.2

In [3]:
dataset = GeneGraphDataset('dataset', 20, 20, graph_num = 100)
data_size = len(dataset)
train_size, test_size = (int)(data_size * train_p), (int)(data_size * test_p)

In [4]:
data_size

2000

In [5]:
dataset = dataset.shuffle()
train_dataset = dataset[:train_size]
test_dataset = dataset[train_size:(train_size + test_size)]
val_dataset = dataset[(train_size + test_size):]

In [6]:
# len(train_dataset), len(test_dataset), len(val_dataset)

In [7]:
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)
val_loader = DataLoader(val_dataset, batch_size=1)

In [8]:
# len(train_loader), len(test_loader), len(val_loader)

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = G2Dist_GCNConv().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay = 0.0001)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10,
                              min_lr=0.00001)

In [10]:
# loss_fn = MSELoss()
# l1_fn = L1Loss()

loss_fn = CrossEntropyLoss()

def train(train_loader):
    model.train()

    total_loss, counter = 0, 0
    size = len(train_loader)
    for batch, data in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.batch)
        #loss = (out.squeeze() - data.y).abs().sum()
        pred, y = out.softmax(axis = 1).argmax(axis = 1), data.y
        counter += (pred == y).sum().item()
        
        loss = loss_fn(out, data.y)
        
        loss.backward()
        total_loss += loss.item()
        optimizer.step()
        
    return total_loss / len(train_loader), counter

In [11]:
@torch.no_grad()
def test(loader):
    model.eval()

    total_error, counter = 0, 0
    for data in loader:
        data = data.to(device)
        out = model(data.x, data.edge_index, data.batch)
        
        pred, y = out.softmax(axis = 1).argmax(axis = 1), data.y
        counter += (pred == y).sum().item()
        
        # total_error += (out.squeeze() - data.y).abs().sum().item()
        
        total_error += loss_fn(out, data.y).item()
        
    return total_error / len(loader), counter

In [12]:
writer = SummaryWriter(log_dir='runs_g2d_10/g2dist_0020_0020_02000-gcn-run10')

In [13]:
import numpy as np
for epoch in range(1, 1001):
    loss, train_counter = train(train_loader)
    test_mae, test_counter = test(test_loader)
    val_mae, _ = test(val_loader)
    
    # scheduler.step(loss)
    
    writer.add_scalar('Loss/train', loss, epoch)
    writer.add_scalar('Loss/test', test_mae, epoch)
    writer.add_scalar('Loss/val', val_mae, epoch)
    writer.add_scalar('Counter/train', train_counter/len(train_loader.dataset), epoch)
    writer.add_scalar('Counter/test', test_counter/len(test_loader.dataset), epoch)
    
    print(f'{time.ctime()}\t'
          f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_mae:.4f}, '
          f'Test: {test_mae:.4f}')
    
    print(f'\t\t -- train_counter: {train_counter}, test_counter:{test_counter}')

Sat Jan  1 22:38:29 2022	Epoch: 001, Loss: 3.0023, Val: 3.3581, Test: 3.2833
		 -- train_counter: 65, test_counter:21
Sat Jan  1 22:38:32 2022	Epoch: 002, Loss: 2.8931, Val: 4.4271, Test: 4.4344
		 -- train_counter: 83, test_counter:22
Sat Jan  1 22:38:35 2022	Epoch: 003, Loss: 2.8606, Val: 5.4239, Test: 5.1078
		 -- train_counter: 112, test_counter:24
Sat Jan  1 22:38:38 2022	Epoch: 004, Loss: 2.8327, Val: 10.0936, Test: 9.6779
		 -- train_counter: 117, test_counter:15
Sat Jan  1 22:38:41 2022	Epoch: 005, Loss: 2.8114, Val: 11.5798, Test: 10.8108
		 -- train_counter: 134, test_counter:23
Sat Jan  1 22:38:44 2022	Epoch: 006, Loss: 2.7775, Val: 8.9466, Test: 8.2702
		 -- train_counter: 159, test_counter:24
Sat Jan  1 22:38:47 2022	Epoch: 007, Loss: 2.7291, Val: 7.1029, Test: 6.8132
		 -- train_counter: 190, test_counter:9
Sat Jan  1 22:38:50 2022	Epoch: 008, Loss: 2.6951, Val: 4.7179, Test: 4.6327
		 -- train_counter: 177, test_counter:20
Sat Jan  1 22:38:53 2022	Epoch: 009, Loss: 2.659

In [14]:
model.eval()

G2Dist_GCNConv(
  (node_emb): Embedding(10000, 2)
  (convs): ModuleList(
    (0): GCNConv(80, 100)
    (1): GCNConv(49, 49)
    (2): GCNConv(24, 24)
    (3): GCNConv(12, 12)
  )
  (pools): ModuleList(
    (0): MaxPool1d(kernel_size=4, stride=2, padding=0, dilation=1, ceil_mode=False)
    (1): MaxPool1d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): MaxPool1d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
  )
  (batch_norms): ModuleList(
    (0): BatchNorm(49)
    (1): BatchNorm(24)
    (2): BatchNorm(12)
    (3): BatchNorm(11)
  )
  (mlp): Sequential(
    (0): Linear(in_features=2, out_features=1, bias=True)
  )
  (lin): Sequential(
    (0): Linear(in_features=440, out_features=200, bias=True)
    (1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): ReLU()
    (3): Dropout(p=0.2, inplace=False)
    (4): Linear(in_features=1

In [15]:
tld0 = list(train_loader)[0].to(device)
tld1 = list(test_loader)[0].to(device)

In [16]:
res0 = model(tld0.x, tld0.edge_index, tld0.batch)

In [17]:
res0

tensor([[ -8.4753,  -7.4071,  -8.9221,  ..., -16.6858,  -4.0825, -10.8624],
        [ -5.8732,  -3.6731,  -2.8419,  ...,  -7.1670, -10.9246,  -7.7702],
        [ 18.4081,  11.3594,   7.4205,  ..., -19.2981, -24.3923, -22.6856],
        ...,
        [ -5.9228,  -3.5704,  -3.0236,  ...,  -6.3995,  -7.5215,  -6.1516],
        [  3.6426,   6.2284,   3.1293,  ..., -14.0113,  -9.8113, -13.2710],
        [ -8.6124,  -6.8912,  -7.3845,  ..., -10.0457,  -5.4492,  -7.9943]],
       device='cuda:0', grad_fn=<AddmmBackward0>)

In [18]:
res0.argmax(axis = 1)

tensor([10,  9,  0,  8, 14, 14, 10,  7,  8, 14,  2,  8,  6, 11, 16, 13,  4,  5,
         1,  7, 13, 13, 12, 10,  7,  6, 14, 14, 11,  5,  2,  1,  2,  3, 12,  6,
         8, 14,  2,  2,  9, 10, 17,  6,  5, 14, 13,  4,  0, 14,  4,  5, 12, 12,
         2,  2, 11, 16,  6,  4, 11,  4, 12, 11, 12,  0,  1, 10,  6, 13,  4, 10,
         0, 11, 13, 12, 10,  7,  4, 15, 14,  7, 12,  8, 13,  8,  5,  0, 12, 12,
         4, 15, 16,  0,  2,  1, 16, 12, 12, 12,  2, 11, 12,  5, 11, 12,  3,  6,
         5, 15, 13, 11, 12, 14,  4, 13, 13,  4,  1, 13,  1,  6, 11, 10,  7,  1,
        13,  3, 13, 17,  9, 15, 16, 12,  2, 14,  5,  5,  4, 14, 11,  0,  3,  7,
         6,  2,  1,  8,  9,  7,  1, 15,  2,  4, 14,  4,  7, 12,  3,  5, 10,  1,
         9, 14,  5, 11,  8, 15, 13,  1,  5, 15, 16,  4, 13, 15, 15,  2,  9, 13,
        11,  3,  2, 16,  4, 17,  2,  2,  6,  9,  4,  8, 11,  0,  4,  8,  4, 15,
        13, 10, 14, 14,  0, 15, 15, 14, 14,  4, 12,  6, 13,  5,  7, 15,  1, 15,
        12, 14,  4, 15,  5, 10,  3, 11, 

In [19]:
tld0.y

tensor([10,  9,  0,  8, 14, 14, 10,  7,  8, 14,  2,  8,  6, 11, 14, 13,  4,  5,
         1,  7, 13, 13, 12, 10,  7,  6, 14, 14, 11,  5,  2,  1,  2,  3, 12,  6,
         8, 14,  2,  2,  9, 10, 17,  6,  5, 14, 14,  4,  0, 14,  4,  5, 12, 12,
         2,  2, 11, 16,  6,  4, 11,  4, 12, 11, 12,  0,  1, 10,  6, 13,  4, 10,
         0, 11, 13,  5, 10,  7,  4, 15, 14,  7, 12,  8, 13,  8,  5,  0, 12,  8,
         1, 15, 16,  0,  2,  1, 10, 10, 12, 12,  2,  6, 12,  5, 11, 12,  3,  6,
         5, 15, 13, 14,  9, 14,  4, 13, 13,  4,  1, 13,  1,  6,  9, 10,  7,  1,
        10,  2, 13, 17,  9, 15, 10, 12,  2,  9,  5,  5,  4, 14, 11,  0,  3,  7,
         6,  2,  1,  8,  9,  7,  1, 15,  2,  4, 14,  4,  7, 12,  1,  5, 10,  1,
         9, 15,  5, 11,  8,  8,  6,  1,  5, 12, 11,  4, 13, 15, 15,  2,  9, 13,
        11,  3,  2, 14,  4, 17,  2,  2,  6,  9,  4,  8, 11,  0,  4,  8,  4, 15,
        13, 10, 11, 14,  1, 15, 13, 14, 14,  4, 12,  6, 13,  5,  7, 15,  1, 15,
        12, 14,  4, 15,  5, 10,  1, 11, 

In [20]:
loss_fn(res0, tld0.y)

tensor(0.4018, device='cuda:0', grad_fn=<NllLossBackward0>)

In [21]:
L1Loss()(res0.argmax(axis = 1).to(torch.float), tld0.y.to(torch.float))

tensor(0.3828, device='cuda:0')

In [22]:
(res0.argmax(axis = 1) - tld0.y).abs().sum().item()/len(tld0.y)

0.3828125

In [23]:
res1 = model(tld1.x, tld1.edge_index, tld1.batch)

In [24]:
res1.argmax(axis = 1)

tensor([ 5,  0, 12,  7, 14,  8, 14, 12,  1, 11,  6,  2,  0, 16,  8,  3, 14, 11,
        16,  0, 15, 12, 15, 13,  2, 12,  8,  3, 15,  9, 15, 16, 12,  6, 13, 13,
         4, 12, 15,  0, 13,  3, 11, 13,  5, 12,  9, 14, 12,  6, 10, 17, 13, 14,
        10, 12,  7,  2, 12,  4, 13, 15, 15, 14], device='cuda:0')

In [25]:
tld1.y

tensor([ 1,  0,  8,  4, 13, 11, 17, 14,  1,  4, 13,  3,  0, 12,  5,  4, 10,  8,
        12,  0, 15, 13, 12, 13,  2,  7,  9,  8, 18, 10, 13, 17, 14,  3,  8, 12,
         2, 17,  8,  0, 16,  6,  7,  5,  4, 14,  9,  8,  5,  4,  8, 12,  9, 11,
        12, 11,  9,  3,  9,  7, 10, 14, 15, 11], device='cuda:0')

In [26]:
loss_fn(res1, tld1.y)

tensor(3.6951, device='cuda:0', grad_fn=<NllLossBackward0>)

In [27]:
L1Loss()(res1.argmax(axis = 1).to(torch.float), tld1.y.to(torch.float))

tensor(2.6562, device='cuda:0')

In [28]:
train_y = [d.y.item() for d in train_dataset]

In [29]:
np.unique(train_y)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19])

In [30]:
test_y = [d.y.item() for d in test_dataset]

In [31]:
np.unique(test_y)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18])

In [32]:
np.unique([d.y.item() for d in val_dataset])

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18])